In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!ls "/content/gdrive/My Drive/"
%cd "/content/gdrive/My Drive/Experiment2/Data"
!unzip "/content/gdrive/My Drive/Experiment2/Data/images_background.zip"
!unzip "/content/gdrive/My Drive/Experiment2/Data/images_evaluation.zip"
%cd "/content/gdrive/My Drive/Experiment2"

In [0]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
from collections import OrderedDict
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [0]:
trainframe=pd.read_csv("train_data.csv")
testframe=pd.read_csv("test_data.csv")

In [0]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [0]:
class Task(object):
  def __init__(self,all_classes,num_classes,num_instances):
    self.all_classes=all_classes
    self.num_classes=num_classes
    self.num_instances=num_instances
    self.train_roots=[]
    self.meta_roots=[]
    self.train_labels=[]
    self.meta_labels=[]
    samples_per_class=20
    sampled_classes=random.sample(all_classes,num_classes)
    label=0
    #labels=list(range(len(sampled_classes)))

    for c in sampled_classes:
      cframe=trainframe.iloc[(c*samples_per_class):((c+1)*samples_per_class)]
      cframe.reset_index(inplace=True,drop=True)
      paths=cframe["Path"]
      sample_idxs=np.random.choice(samples_per_class,samples_per_class,replace=False)
      train_idxs=sample_idxs[:num_instances]
      meta_idxs=sample_idxs[num_instances:(num_instances*2)]
      for idx in train_idxs:
        self.train_roots.append(paths[idx])
        self.train_labels.append(label)
      for idx in meta_idxs:
        self.meta_roots.append(paths[idx])
        self.meta_labels.append(label)
      label+=1

class TestTask(object):
  def __init__(self,all_classes,num_classes,num_instances,num_test_instances):
    self.all_classes=all_classes
    self.num_classes=num_classes
    self.num_instances=num_instances
    self.num_test_instances=num_test_instances
    self.test_roots=[]
    self.train_roots=[]
    self.test_labels=[]
    self.train_labels=[]
    samples_per_class=20
    sampled_classes=random.sample(all_classes,num_classes)
    label=0
    #labels=list(range(len(sampled_classes)))

    for c in sampled_classes:
      cframe=testframe.iloc[((c-964)*samples_per_class):(((c+1)-964)*samples_per_class)]
      cframe.reset_index(inplace=True,drop=True)
      paths=cframe["Path"]
      sample_idxs=np.random.choice(samples_per_class,samples_per_class,replace=False)
      train_idxs=sample_idxs[:num_instances]
      test_idxs=sample_idxs[num_instances:(num_instances+num_test_instances)]
      for idx in test_idxs:
        self.test_roots.append(paths[idx])
        self.test_labels.append(label)
      # for idx in meta_idxs:
      #   self.meta_roots.append(paths[idx])
      #   self.meta_labels.append(label)
      for idx in train_idxs:
        self.train_roots.append(paths[idx])
        self.train_labels.append(label)
      label+=1

In [0]:
class MiniSet(Dataset):
  def __init__(self,fileroots,labels,transform):
    self.fileroots=fileroots
    self.labels=labels
    self.transform=transform

  def __len__(self):
    return len(self.fileroots)

  def __getitem__(self,idx):
    img=Image.open(self.fileroots[idx])
    img=self.transform(img)
    return img,self.labels[idx]

In [0]:
transform=transforms.Compose([transforms.Resize((28,28)),transforms.ToTensor()])
def get_loaders(task):
  loaders={}
  train_fileroots=task.train_roots
  train_labels=task.train_labels
  meta_fileroots=task.meta_roots
  meta_labels=task.meta_labels
  trainloader=DataLoader(MiniSet(train_fileroots,train_labels,transform),
                         batch_size=len(train_fileroots),shuffle=True)
  metaloader=DataLoader(MiniSet(meta_fileroots,meta_labels,transform),
                         batch_size=len(meta_fileroots),shuffle=True)
  loaders["train"]=trainloader
  loaders["meta"]=metaloader
  return loaders

def get_test_loaders(task):
  loaders={}
  test_fileroots=task.test_roots
  test_labels=task.test_labels
  train_fileroots=task.train_roots
  train_labels=task.train_labels
  testloader=DataLoader(MiniSet(test_fileroots,test_labels,transform),
                         batch_size=len(test_fileroots),shuffle=True)
  # metaloader=DataLoader(MiniSet(meta_fileroots,meta_labels,transform),
  #                        batch_size=len(meta_fileroots),shuffle=True)
  trainloader=DataLoader(MiniSet(train_fileroots,train_labels,transform),
                         batch_size=len(train_fileroots),shuffle=True)
  loaders["train"]=trainloader
  #loaders["meta"]=metaloader
  loaders["test"]=testloader
  return loaders

In [0]:
class BaseNet(nn.Module):
  def __init__(self,num_classes):
    super(BaseNet,self).__init__()
    self.features = nn.Sequential(OrderedDict([
                ('conv1', nn.Conv2d(1, 64, 3)),
                ('bn1', nn.BatchNorm2d(64, momentum=1, affine=True)),
                ('relu1', nn.ReLU(inplace=True)),
                ('pool1', nn.MaxPool2d(2,2)),
                ('conv2', nn.Conv2d(64,64,3)),
                ('bn2', nn.BatchNorm2d(64, momentum=1, affine=True)),
                ('relu2', nn.ReLU(inplace=True)),
                ('pool2', nn.MaxPool2d(2,2)),
                ('conv3', nn.Conv2d(64,64,3)),
                ('bn3', nn.BatchNorm2d(64, momentum=1, affine=True)),
                ('relu3', nn.ReLU(inplace=True)),
                ('pool3', nn.MaxPool2d(2,2))]))    
    self.add_module('fc', nn.Linear(64,num_classes))

  def forward(self,x,weights=None):
    if weights==None:  
      output=self.features(x)
      output=output.view(-1,64)
      output=self.fc(output)
    else:
      x = F.conv2d(x, weights['meta_learner.features.conv1.weight'], weights['meta_learner.features.conv1.bias'])
      x = F.batch_norm(x, weights['meta_learner.features.bn1.running_mean'], 
                         weights['meta_learner.features.bn1.running_var'],
                          weights['meta_learner.features.bn1.weight'],
                         weights['meta_learner.features.bn1.bias'],momentum=1,training=True)
      x = F.relu(x)
      x = F.max_pool2d(x, kernel_size=2, stride=2) 
      x = F.conv2d(x, weights['meta_learner.features.conv2.weight'], weights['meta_learner.features.conv2.bias'])
      x = F.batch_norm(x, weights['meta_learner.features.bn2.running_mean'], 
                         weights['meta_learner.features.bn2.running_var'],
                          weights['meta_learner.features.bn2.weight'],
                         weights['meta_learner.features.bn2.bias'],momentum=1,training=True)
      x = F.relu(x)
      x = F.max_pool2d(x, kernel_size=2, stride=2) 
      x = F.conv2d(x, weights['meta_learner.features.conv3.weight'], weights['meta_learner.features.conv3.bias'])
      x = F.batch_norm(x, weights['meta_learner.features.bn3.running_mean'], 
                         weights['meta_learner.features.bn3.running_var'],
                          weights['meta_learner.features.bn3.weight'],
                         weights['meta_learner.features.bn3.bias'],momentum=1,training=True)
      x = F.relu(x)
      x = F.max_pool2d(x, kernel_size=2, stride=2) 
      x = x.view(x.size(0), 64)
      output = F.linear(x, weights['meta_learner.fc.weight'], weights['meta_learner.fc.bias'])
    out = F.log_softmax(output, dim=1)
    return out

In [0]:
class MetaLearner(nn.Module):
  def __init__(self,num_classes):
    super(MetaLearner, self).__init__()
    #self.params=params
    self.meta_learner=BaseNet(num_classes)

  def forward(self,x,mod_weights=None):
    if mod_weights==None:
      out=self.meta_learner(x)
    else:
      out=self.meta_learner(x,mod_weights)
    return out
  
  def clone_state_dict(self):
    cloned_state_dict = {key: val.clone()for key, val in self.state_dict().items()}
    return cloned_state_dict


In [0]:
def train_single_task(net,lr,loaders,num_updates,loss_metric):
  net.train()
  trainloader=loaders["train"]
  x,y=trainloader.__iter__().next()
  x.to(device)
  y.to(device)
  output=net(x)
  loss=loss_metric(output,y)

  def zero_grad(params):
    for p in params:
      if p.grad is not None:
        p.grad.zero_()
  zero_grad(net.parameters())
  grads=torch.autograd.grad(loss,net.parameters(),create_graph=True)
  mod_state_dict=net.clone_state_dict()
  mod_weights=OrderedDict()
  for (k,v),g in zip(net.named_parameters(),grads):
    mod_weights[k]=v-lr*g
    mod_state_dict[k]=mod_weights[k]
  for i in range(1,num_updates):
    output=net(x,mod_state_dict)
    loss=loss_metric(output,y)
    zero_grad(mod_weights.values())
    grads=torch.autograd.grad(loss,mod_weights.values(),create_graph=True)
    for (k,v),g in zip(mod_weights.items(),grads):
      mod_weights[k]=v-lr*g
      mod_state_dict[k]=mod_weights[k]

  return mod_state_dict

In [0]:
def train(net,meta_train_classes,meta_optimiser,loss_metric,num_classes,num_instances,num_tasks,lr,meta_lr,num_inner_updates,num_epochs):
  total_loss=0  
  print_every=100
  plot_every=2
  meta_losses=[]
  for epoch in range(1,num_epochs+1):
    state_dicts=[]
    loaders_list=[]
    for n in range(num_tasks):
      task=Task(meta_train_classes,num_classes,num_instances)
      loaders=get_loaders(task)
      d=train_single_task(net,lr,loaders,num_inner_updates,loss_metric)
      state_dicts.append(d)
      loaders_list.append(loaders)
    metaloss=0
    for n in range(num_tasks):
      loaders=loaders_list[n]
      metaloader=loaders["meta"]
      x,y=metaloader.__iter__().next()
      x.to(device)
      y.to(device)
      d=state_dicts[n]
      output=net(x,d)
      loss=loss_metric(output,y)
      metaloss+=loss
    metaloss/=float(num_tasks)
    total_loss+=metaloss.item()
    meta_optimiser.zero_grad()
    metaloss.backward()
    meta_optimiser.step()
    if epoch % print_every == 0:
      print("{}/{}. loss: {}".format(epoch, num_epochs, total_loss / plot_every))
    if epoch%plot_every==0:
      meta_losses.append(total_loss/plot_every)
      total_loss = 0
    if (epoch%20)==0:
      print("Epoch "+str(epoch)+" completed.")
  return meta_losses,net

In [0]:
loss_metric=nn.NLLLoss()
num_classes=5
net=MetaLearner(num_classes)
lr=1e-1
meta_lr=1e-3
meta_optimizer = torch.optim.Adam(net.parameters(), lr=meta_lr)
num_instances=3
num_tasks=10
num_inner_updates=1
num_epochs=1000
train_classes=np.max(trainframe['Label'])
train_classes=list(np.arange(train_classes))
metalosses,net=train(net,train_classes,meta_optimizer,loss_metric,num_classes,num_instances,num_tasks,lr,meta_lr,num_inner_updates,num_epochs)

In [0]:
plt.plot(metalosses)

In [0]:
def accuracy(outputs, labels):
  outputs = np.argmax(outputs, axis=1)
  return np.sum(outputs == labels) / float(labels.size)


In [0]:
def evaluate(net,test_classes,task_lr,num_classes=5,num_steps=100,num_eval_updates=3):
  #net.eval()
  losses=[]
  acc_list=[]
  for step in np.arange(num_steps):
    task=TestTask(test_classes,num_classes=5,num_instances=3,num_test_instances=10)
    loaders=get_test_loaders(task)
    trainloader,testloader=loaders["train"],loaders["test"]
    x_train,y_train=trainloader.__iter__().next()
    x_test,y_test=testloader.__iter__().next()
    x_train.to(device)
    y_train.to(device)
    x_test.to(device)
    y_test.to(device)
    #def __init__(self,all_classes,num_classes,num_instances,num_test_instances)
    cloned_net=copy.deepcopy(net)
    optim = torch.optim.SGD(cloned_net.parameters(),lr=task_lr)
    for _ in range(num_eval_updates):
      y_train_pred=cloned_net(x_train)
      loss=loss_metric(y_train_pred,y_train)
      optim.zero_grad()
      loss.backward()
      optim.step()
    y_test_pred=cloned_net(x_test)
    loss=loss_metric(y_test_pred,y_test)
    losses.append(loss)
    y_test_pred=y_test_pred.data.cpu().numpy()
    y_test=y_test.data.cpu().numpy()
    #print(y_test)
    #print(y_test_pred)
    acc=accuracy(y_test_pred,y_test)
    acc_list.append(acc)
  return acc_list,losses

In [0]:
import copy
#test_classes=np.max(testframe['Label'])-np.min(testframe['Label'])
test_classes=list(np.arange(np.min(testframe['Label']),np.max(testframe['Label']+1)))
acc_list,losses=evaluate(net,test_classes,task_lr=1e-1)


In [0]:
average_test_accuracy=(sum(acc_list)/len(acc_list))

In [0]:
average_test_accuracy